In [15]:
import sys, os
import requests
import selenium
from selenium import webdriver
import requests
from pandas import DataFrame
from bs4 import BeautifulSoup
import re
from datetime import datetime
import pickle, json, glob, time
from tqdm import tqdm



###### 날짜 저장 ##########
date = str(datetime.now())
date = date[:date.rfind(':')].replace(' ', '_')
date = date.replace(':','시') + '분'

sleep_sec = 0.5


####### 언론사별 본문 위치 태그 파싱 함수 ###########
print('본문 크롤링에 필요한 함수를 로딩하고 있습니다...\n' + '-' * 100)
def crawling_main_text(url):

    req = requests.get(url)
    req.encoding = None
    soup = BeautifulSoup(req.text, 'html.parser')
    
    # 연합뉴스
    if ('://yna' in url) | ('app.yonhapnews' in url): 
        main_article = soup.find('article', {'class':'story-news article'})
        if main_article == None:
            main_article = soup.find('div', {'class' : 'article-txt'})
            
        text = main_article.text
        
    # MBC 
    elif '//imnews.imbc' in url: 
        text = soup.find('div', {'itemprop' : 'articleBody'}).text
        
    # 매일경제(미라클), req.encoding = None 설정 필요
    elif 'mirakle.mk' in url:
        text = soup.find('div', {'class' : 'view_txt'}).text
        
    # 매일경제, req.encoding = None 설정 필요
    elif 'mk.co' in url:
        text = soup.find('div', {'class' : 'art_txt'}).text
        
    # SBS
    elif 'news.sbs' in url:
        text = soup.find('div', {'itemprop' : 'articleBody'}).text
    
    # KBS
    elif 'news.kbs' in url:
        text = soup.find('div', {'id' : 'cont_newstext'}).text
        
    # JTBC
    elif 'news.jtbc' in url:
        text = soup.find('div', {'class' : 'article_content'}).text
        
    # 그 외
    else:
        text == None
        
    return text.replace('\n','').replace('\r','').replace('<br>','').replace('\t','')
    
    
press_nm = 'JTBC'

print('검색할 언론사 : {}'.format(press_nm))


############### 브라우저를 켜고 검색 키워드 입력 ####################
query = input('검색할 키워드  : ')
news_num = int(input('수집 뉴스의 수(숫자만 입력) : '))

print('\n' + '=' * 100 + '\n')

print('브라우저를 실행시킵니다(자동 제어)\n')
chrome_path = 'C:/Users/jhjh3/chromedriver_win32 (1)/chromedriver.exe'
browser = webdriver.Chrome(chrome_path)

news_url = 'https://search.naver.com/search.naver?where=news&query={}'.format(query)
browser.get(news_url)
time.sleep(sleep_sec)


######### 언론사 선택 및 confirm #####################
print('설정한 언론사를 선택합니다.\n')

search_opn_btn = browser.find_element_by_xpath('//a[@class="btn_option _search_option_open_btn"]')
search_opn_btn.click()
time.sleep(sleep_sec)

bx_press = browser.find_element_by_xpath('//div[@role="listbox" and @class="api_group_option_sort _search_option_detail_wrap"]//li[@class="bx press"]')

# 기준 두번 째(언론사 분류순) 클릭하고 오픈하기
press_tablist = bx_press.find_elements_by_xpath('.//div[@role="tablist" and @class="option"]/a')
press_tablist[1].click()
time.sleep(sleep_sec)

# 첫 번째 것(언론사 분류선택)
bx_group = bx_press.find_elements_by_xpath('.//div[@class="api_select_option type_group _category_select_layer"]/div[@class="select_wrap _root"]')[0]

press_kind_bx = bx_group.find_elements_by_xpath('.//div[@class="group_select _list_root"]')[0]
press_kind_btn_list = press_kind_bx.find_elements_by_xpath('.//ul[@role="tablist" and @class="lst_item _ul"]/li/a')


for press_kind_btn in press_kind_btn_list:
    
    # 언론사 종류를 순차적으로 클릭(좌측)
    press_kind_btn.click()
    time.sleep(sleep_sec)
    
    # 언론사선택(우측)
    press_slct_bx = bx_group.find_elements_by_xpath('.//div[@class="group_select _list_root"]')[1]
    # 언론사 선택할 수 있는 클릭 버튼
    press_slct_btn_list = press_slct_bx.find_elements_by_xpath('.//ul[@role="tablist" and @class="lst_item _ul"]/li/a')
    # 언론사 이름들 추출
    press_slct_btn_list_nm = [psl.text for psl in press_slct_btn_list]
    
    # 언론사 이름 : 언론사 클릭 버튼 인 딕셔너리 생성
    press_slct_btn_dict = dict(zip(press_slct_btn_list_nm, press_slct_btn_list))
    
    # 원하는 언론사가 해당 이름 안에 있는 경우
    # 1) 클릭하고
    # 2) 더이상 언론사분류선택 탐색 중지
    if press_nm in press_slct_btn_dict.keys():
        print('<{}> 카테고리에서 <{}>를 찾았으므로 탐색을 종료합니다'.format(press_kind_btn.text, press_nm))
        
        press_slct_btn_dict[press_nm].click()
        time.sleep(sleep_sec)
        
        break



################ 뉴스 크롤링 ########################

print('\n크롤링을 시작합니다.')
# ####동적 제어로 페이지 넘어가며 크롤링
news_dict = {}
idx = 1
cur_page = 1

pbar = tqdm(total=news_num ,leave = True)
    
while idx < news_num:

    table = browser.find_element_by_xpath('//ul[@class="list_news"]')
    li_list = table.find_elements_by_xpath('./li[contains(@id, "sp_nws")]')
    area_list = [li.find_element_by_xpath('.//div[@class="news_area"]') for li in li_list]
    a_list = [area.find_element_by_xpath('.//a[@class="news_tit"]') for area in area_list]
 
    for n in a_list[:min(len(a_list), news_num-idx+1)]:
        n_url = n.get_attribute('href')
        news_dict[idx] = {'title' : n.get_attribute('title'), 
                          'url' : n_url,
                          'text' : crawling_main_text(n_url)}
        
        idx += 1
        pbar.update(1)
        
    if idx < news_num:
        cur_page +=1

        pages = browser.find_element_by_xpath('//div[@class="sc_page_inner"]')
        next_page_url = [p for p in pages.find_elements_by_xpath('.//a') if p.text == str(cur_page)][0].get_attribute('href')

        browser.get(next_page_url)
        time.sleep(sleep_sec)
    else:
        pbar.close()
        
        print('\n브라우저를 종료합니다.\n' + '=' * 100)
        time.sleep(0.7)
        browser.close()
        break

#### 데이터 전처리하기 ###################################################### 

print('데이터프레임 변환\n')
news_df = DataFrame(news_dict).T

folder_path = os.getcwd()
xlsx_file_name = '네이버뉴스_본문_{}개_{}_{}.xlsx'.format(news_num, query, date)

news_df.to_excel(xlsx_file_name)

print('엑셀 저장 완료 | 경로 : {}\\{}\n'.format(folder_path, xlsx_file_name))

os.startfile(folder_path)

print('=' * 100 + '\n결과물의 일부')
news_df

본문 크롤링에 필요한 함수를 로딩하고 있습니다...
----------------------------------------------------------------------------------------------------
검색할 언론사 : JTBC
검색할 키워드  : 코로나
수집 뉴스의 수(숫자만 입력) : 3


브라우저를 실행시킵니다(자동 제어)



C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_1488/3928598573.py:83: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(chrome_path)


설정한 언론사를 선택합니다.



C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_1488/3928598573.py:93: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_opn_btn = browser.find_element_by_xpath('//a[@class="btn_option _search_option_open_btn"]')
C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_1488/3928598573.py:97: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  bx_press = browser.find_element_by_xpath('//div[@role="listbox" and @class="api_group_option_sort _search_option_detail_wrap"]//li[@class="bx press"]')
C:\Users\jhjh3\anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py:426: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")


<방송/통신> 카테고리에서 <JTBC>를 찾았으므로 탐색을 종료합니다

크롤링을 시작합니다.


  0%|                                                                                            | 0/3 [00:00<?, ?it/s]C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_1488/3928598573.py:152: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  table = browser.find_element_by_xpath('//ul[@class="list_news"]')
C:\Users\jhjh3\anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py:393: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")
100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:03<00:00,  1.12s/it]



브라우저를 종료합니다.
데이터프레임 변환

엑셀 저장 완료 | 경로 : C:\Users\jhjh3\PJT_02\네이버뉴스_본문_3개_코로나_2022-03-28_09시46분.xlsx

결과물의 일부


,title,url,text
1,"""코로나 확산세 꺾였다""…이번주 거리두기 완화 여부 '주목'",https://news.jtbc.joins.com/article/article.as...,지난 27일 오전 서울역 광장에 설치된 코로나19 임시선별 검사소를 찾은 시민들이 ...
2,코로나19 위중증 다시 1200명대로…신규 확진 나흘째 30만명대,https://news.jtbc.joins.com/article/article.as...,지난 26일 오전 중구 서울역 광장 코로나19 임시 선별검사소를 찾은 시민들이 검사...
3,"코로나 확진 산모, 2시간 넘게 병원 찾다 구급차서 출산",https://news.jtbc.joins.com/article/article.as...,〈자료사진=JTBC 방송화면 캡처〉 인천에서 신종 코로나바이러스 감염증...


In [14]:
# news_list = []

import os
import sys
import requests
import json
import urllib.request
import pandas as pd

len(news_df.title)
news_list=[]

client_id = "qj40wl5n0t"
client_secret = "1xzXEYgwlsbv9RQxd1onUSByiyvtBDPFV9NEY1mv"

for i in range(len(news_df.title)):
    headers = {
    "X-NCP-APIGW-API-KEY-ID": client_id,
    "X-NCP-APIGW-API-KEY": client_secret,
    "Content-Type": "application/json"
}
    language = "ko" # Language of document (ko, ja )
    model = "news" # Model used for summaries (general, news)
    tone = "2" # Converts the tone of the summarized result. (0, 1, 2, 3)
    summaryCount = "3" # This is the number of sentences for the summarized document.
    url= "https://naveropenapi.apigw.ntruss.com/text-summary/v1/summarize" 
    title= ""
    content = news_df.text[i+1]
    data = {
        "document": {
        "title": title,
        "content" : content
        },
        "option": {
        "language": language,
        "model": model,
        "tone": tone,
        "summaryCount" : summaryCount
        }
    }
    # print(json.dumps(data, indent=4, sort_keys=True))
    response = requests.post(url, data=json.dumps(data), headers=headers)
    rescode = response.status_code
    if(rescode == 200):
        r1=response.text
        print(r1)

    else:
        print("Error : " + response.text)
        
    r1

    #r1 = json.loads(r1)
    r1d = json.loads(r1)
        # r1 = json.dumps(r1)
        # r1r1= r1['summary']
    r1r1 = r1d['summary']
#    r1r1= r1.split('연합뉴스')[1]
    r1r1 = '{}번째 뉴스입니다.{}'.format(i+1, r1r1)
    news_list.append(r1r1)
    print(r1r1)
    print(news_list)
    
    encText = urllib.parse.quote(r1r1)
    data = "speaker=njihun&volume=0&speed=0&pitch=0&format=mp3&text=" + encText;
    url = "https://naveropenapi.apigw.ntruss.com/tts-premium/v1/tts"
    request = urllib.request.Request(url)
    request.add_header("X-NCP-APIGW-API-KEY-ID",client_id)
    request.add_header("X-NCP-APIGW-API-KEY",client_secret)
    response = urllib.request.urlopen(request, data=data.encode('utf-8'))
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        fn = 'voice{}.mp3'.format(i)
        with open(fn, 'wb') as f:
            f.write(response_body)
        print(fn)
    else:
        print("Error Code:" + rescode)

Error : {"status":400,"error":{"errorCode":"E003","message":"Text quota Exceeded"}}
1번째 뉴스입니다.사진 제공: 연합뉴스우크라이나-폴란드 국경을 방문한 조 바이든 미국 대통령이 오늘 폴란드 대통령과 회담합니다.
폴란드는 이번 전쟁의 난민을 가장 많이 수용한 국가로, 폴란드 지원 방안 등이 논의될 것이라고 외신들은 보도하고 있습니다.
우크라이나 전쟁 발발 이후 처음으로 유럽을 방문한 바이든 대통령은 어제 우크라이나 국경에서 80㎞ 떨어진 미군 부대를 방문하고 피난민을 돕는 구호단체 관계자들과 간담회를 했습니다.
['1번째 뉴스입니다.사진 제공: 연합뉴스우크라이나-폴란드 국경을 방문한 조 바이든 미국 대통령이 오늘 폴란드 대통령과 회담합니다.\n폴란드는 이번 전쟁의 난민을 가장 많이 수용한 국가로, 폴란드 지원 방안 등이 논의될 것이라고 외신들은 보도하고 있습니다.\n우크라이나 전쟁 발발 이후 처음으로 유럽을 방문한 바이든 대통령은 어제 우크라이나 국경에서 80㎞ 떨어진 미군 부대를 방문하고 피난민을 돕는 구호단체 관계자들과 간담회를 했습니다.']
voice0.mp3
Error : {"status":400,"error":{"errorCode":"E003","message":"Text quota Exceeded"}}
2번째 뉴스입니다.사진 제공: 연합뉴스우크라이나-폴란드 국경을 방문한 조 바이든 미국 대통령이 오늘 폴란드 대통령과 회담합니다.
폴란드는 이번 전쟁의 난민을 가장 많이 수용한 국가로, 폴란드 지원 방안 등이 논의될 것이라고 외신들은 보도하고 있습니다.
우크라이나 전쟁 발발 이후 처음으로 유럽을 방문한 바이든 대통령은 어제 우크라이나 국경에서 80㎞ 떨어진 미군 부대를 방문하고 피난민을 돕는 구호단체 관계자들과 간담회를 했습니다.
['1번째 뉴스입니다.사진 제공: 연합뉴스우크라이나-폴란드 국경을 방문한 조 바이든 미국 대통령이 오늘 폴란드 대통령과 회담합니다.\n폴란드는 이번 전쟁의 난민

In [13]:
# news_list = []

import os
import sys
import requests
import json
import urllib.request
import pandas as pd

len(news_df.title)
news_list=[]

client_id = "qj40wl5n0t"
client_secret = "1xzXEYgwlsbv9RQxd1onUSByiyvtBDPFV9NEY1mv"

for i in range(len(news_df.title)):
    headers = {
    "X-NCP-APIGW-API-KEY-ID": client_id,
    "X-NCP-APIGW-API-KEY": client_secret,
    "Content-Type": "application/json"
}
    language = "ko" # Language of document (ko, ja )
    model = "news" # Model used for summaries (general, news)
    tone = "2" # Converts the tone of the summarized result. (0, 1, 2, 3)
    summaryCount = "3" # This is the number of sentences for the summarized document.
    url= "https://naveropenapi.apigw.ntruss.com/text-summary/v1/summarize" 
    title= ""
    content = news_df.text[i+1]
    data = {
        "document": {
        "title": title,
        "content" : content
        },
        "option": {
        "language": language,
        "model": model,
        "tone": tone,
        "summaryCount" : summaryCount
        }
    }
    # print(json.dumps(data, indent=4, sort_keys=True))
    response = requests.post(url, data=json.dumps(data), headers=headers)
    rescode = response.status_code
    if(rescode == 200):
        r1=response.text
        print(r1)

    else:
        print("Error : " + response.text)
        
    r1

Error : {"status":400,"error":{"errorCode":"E003","message":"Text quota Exceeded"}}
Error : {"status":400,"error":{"errorCode":"E003","message":"Text quota Exceeded"}}
{"summary":"사진 제공: 연합뉴스우크라이나-폴란드 국경을 방문한 조 바이든 미국 대통령이 오늘 폴란드 대통령과 회담합니다.\n폴란드는 이번 전쟁의 난민을 가장 많이 수용한 국가로, 폴란드 지원 방안 등이 논의될 것이라고 외신들은 보도하고 있습니다.\n우크라이나 전쟁 발발 이후 처음으로 유럽을 방문한 바이든 대통령은 어제 우크라이나 국경에서 80㎞ 떨어진 미군 부대를 방문하고 피난민을 돕는 구호단체 관계자들과 간담회를 했습니다."}


In [6]:
fn

'voice2.mp3'

In [8]:
r1r1

'3번째 뉴스입니다.김은혜 당선인 대변인 [사진 제공: 연합뉴스]윤석열 당선인 측이 문재인 대통령을 겨냥해 "윤석열 당선인의 판단에 마치 문제가 있고 참모들이 당선인의 판단을 흐리는 것처럼 언급하신 것은 대단히 유감스럽다"고 밝혔습니다.\n김은혜 당선인 대변인은 서면 브리핑을 통해 "박수현 소통수석을 통해 전달된 문재인 대통령의 말씀과 관련해 문의가 많아 말씀드린다"며 이같이 말했습니다.\n김 대변인은 "정부 인수인계가 원활치 않은 상황에서, 코로나19와 경제위기 대응이 긴요한 때에 두 분의 만남을 \'덕담 나누는 자리\' 정도로 평가하는 것에 대해서도 쉽게 동의하기 어렵다"고 지적했습니다.'

In [11]:
audio1 = open("voice0.mp3", "rb").read()
audio2 = open("voice1.mp3", "rb").read()
audio3 = open("voice2.mp3", "rb").read()
audioJoin = audio1 + audio2 + audio3
audioFinal = open("audioFinal.mp3", "wb").write(audioJoin)

In [10]:
audioFinal

1000224

In [26]:
news_list

NameError: name 'news_list' is not defined

In [22]:
import os
import sys
import requests
import json
import urllib.request
# import pandas as pd
# news_df = pd.read_excel('./news_df.xlsx')

def process_b(news_df):
    len(news_df.title)
    client_id = "qj40wl5n0t"
    client_secret = "1xzXEYgwlsbv9RQxd1onUSByiyvtBDPFV9NEY1mv"

    for i in range(len(news_df.title)):
        headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret,
        "Content-Type": "application/json"
    }
        language = "ko" # Language of document (ko, ja )
        model = "news" # Model used for summaries (general, news)
        tone = "2" # Converts the tone of the summarized result. (0, 1, 2, 3)
        summaryCount = "3" # This is the number of sentences for the summarized document.
        url= "https://naveropenapi.apigw.ntruss.com/text-summary/v1/summarize" 
        title= ""
        content = news_df.text[i+1]
        data = {
            "document": {
            "title": title,
            "content" : content
            },
            "option": {
            "language": language,
            "model": model,
            "tone": tone,
            "summaryCount" : summaryCount
            }
        }
        # print(json.dumps(data, indent=4, sort_keys=True))
        response = requests.post(url, data=json.dumps(data), headers=headers)
        rescode = response.status_code
        if(rescode == 200):
            r1=response.text  
            # print(r1)

        else:
            print("Error : " + response.text)
    
        # r1 = json.loads(r1)
        r1d = json.loads(r1)
        # r1 = json.dumps(r1)
        # r1r1= r1['summary']
        r1r1 = r1d['summary']
        #r1r1= r1.split('연합뉴스')[1]
        r1r1 = '{}번째 뉴스입니다.{}'.format(i+1, r1r1)
        print(r1r1)
    
        encText = urllib.parse.quote(r1r1)
        data = "speaker=njihun&volume=0&speed=0&pitch=0&format=mp3&text=" + encText;
        url = "https://naveropenapi.apigw.ntruss.com/tts-premium/v1/tts"
        request = urllib.request.Request(url)
        request.add_header("X-NCP-APIGW-API-KEY-ID",client_id)
        request.add_header("X-NCP-APIGW-API-KEY",client_secret)
        response = urllib.request.urlopen(request, data=data.encode('utf-8'))
        rescode = response.getcode()
        if(rescode==200):
            response_body = response.read()
            fn = 'voice{}.mp3'.format(i)
            with open(fn, 'wb') as f:
                f.write(response_body)
            print(fn)
        else:
            print("Error Code:" + rescode)
            
    result = (r1r1, fn)
    return result


In [37]:
result = process_b(news_df)
str(result)

1번째 뉴스입니다.북한이 대륙간 탄도미사일, ICBM을 시험 발사한 것과 관련해 문재인 대통령은 "김정은 국무위원장이 국제사회에 약속한 발사 유예를 스스로 파기한 것"이라며 "강력히 규탄한다"고 밝혔습니다.
문 대통령은 오늘 오후 청와대에서 국가안전보장회의, NSC 회의를 긴급 소집해 "북한의 이번 발사는 한반도와 지역·국제사회에 심각한 위협을 야기하고 유엔안보리 결의를 명백히 위반한 것"이라며 이같이 말했습니다.
이어 "정부 교체기 안보에 한 치의 빈틈도 없도록 한미동맹을 바탕으로 국제사회와 협력해 모든 대응 조치를 강구하라"고 지시하고, 특히 "대통령 당선인 측과도 긴밀하게 협력하라"고 당부했습니다.
voice0.mp3
2번째 뉴스입니다.사진 제공: 연합뉴스]문재인 대통령이 박근혜 전 대통령에게 '늘 건강하십시오'라는 문구가 적힌 퇴원 축하 난을 보냈다고, 청와대가 밝혔습니다.
또 축하난을 받은 박 전 대통령은 문 대통령에게 "마무리 잘 하시고, 건강을 잘 챙기시길 바란다"는 말을 전해 왔다고 청와대는 설명했습니다.
청와대는 "문 대통령의 축하난은 지난 22일 오후, 김한규 청와대 정무비서관이 유영하 변호사를 통해 박 전 대통령에게 전달했다"고 덧붙였습니다.
voice1.mp3
3번째 뉴스입니다.김은혜 당선인 대변인 [사진 제공: 연합뉴스]윤석열 당선인 측이 문재인 대통령을 겨냥해 "윤석열 당선인의 판단에 마치 문제가 있고 참모들이 당선인의 판단을 흐리는 것처럼 언급하신 것은 대단히 유감스럽다"고 밝혔습니다.
김은혜 당선인 대변인은 서면 브리핑을 통해 "박수현 소통수석을 통해 전달된 문재인 대통령의 말씀과 관련해 문의가 많아 말씀드린다"며 이같이 말했습니다.
김 대변인은 "정부 인수인계가 원활치 않은 상황에서, 코로나19와 경제위기 대응이 긴요한 때에 두 분의 만남을 '덕담 나누는 자리' 정도로 평가하는 것에 대해서도 쉽게 동의하기 어렵다"고 지적했습니다.
voice2.mp3


'(\'3번째 뉴스입니다.김은혜 당선인 대변인 [사진 제공: 연합뉴스]윤석열 당선인 측이 문재인 대통령을 겨냥해 "윤석열 당선인의 판단에 마치 문제가 있고 참모들이 당선인의 판단을 흐리는 것처럼 언급하신 것은 대단히 유감스럽다"고 밝혔습니다.\\n김은혜 당선인 대변인은 서면 브리핑을 통해 "박수현 소통수석을 통해 전달된 문재인 대통령의 말씀과 관련해 문의가 많아 말씀드린다"며 이같이 말했습니다.\\n김 대변인은 "정부 인수인계가 원활치 않은 상황에서, 코로나19와 경제위기 대응이 긴요한 때에 두 분의 만남을 \\\'덕담 나누는 자리\\\' 정도로 평가하는 것에 대해서도 쉽게 동의하기 어렵다"고 지적했습니다.\', \'voice2.mp3\')'

In [25]:
process_b(news_df)

1번째 뉴스입니다.북한이 대륙간 탄도미사일, ICBM을 시험 발사한 것과 관련해 문재인 대통령은 "김정은 국무위원장이 국제사회에 약속한 발사 유예를 스스로 파기한 것"이라며 "강력히 규탄한다"고 밝혔습니다.
문 대통령은 오늘 오후 청와대에서 국가안전보장회의, NSC 회의를 긴급 소집해 "북한의 이번 발사는 한반도와 지역·국제사회에 심각한 위협을 야기하고 유엔안보리 결의를 명백히 위반한 것"이라며 이같이 말했습니다.
이어 "정부 교체기 안보에 한 치의 빈틈도 없도록 한미동맹을 바탕으로 국제사회와 협력해 모든 대응 조치를 강구하라"고 지시하고, 특히 "대통령 당선인 측과도 긴밀하게 협력하라"고 당부했습니다.
voice0.mp3
2번째 뉴스입니다.사진 제공: 연합뉴스]문재인 대통령이 박근혜 전 대통령에게 '늘 건강하십시오'라는 문구가 적힌 퇴원 축하 난을 보냈다고, 청와대가 밝혔습니다.
또 축하난을 받은 박 전 대통령은 문 대통령에게 "마무리 잘 하시고, 건강을 잘 챙기시길 바란다"는 말을 전해 왔다고 청와대는 설명했습니다.
청와대는 "문 대통령의 축하난은 지난 22일 오후, 김한규 청와대 정무비서관이 유영하 변호사를 통해 박 전 대통령에게 전달했다"고 덧붙였습니다.
voice1.mp3
3번째 뉴스입니다.김은혜 당선인 대변인 [사진 제공: 연합뉴스]윤석열 당선인 측이 문재인 대통령을 겨냥해 "윤석열 당선인의 판단에 마치 문제가 있고 참모들이 당선인의 판단을 흐리는 것처럼 언급하신 것은 대단히 유감스럽다"고 밝혔습니다.
김은혜 당선인 대변인은 서면 브리핑을 통해 "박수현 소통수석을 통해 전달된 문재인 대통령의 말씀과 관련해 문의가 많아 말씀드린다"며 이같이 말했습니다.
김 대변인은 "정부 인수인계가 원활치 않은 상황에서, 코로나19와 경제위기 대응이 긴요한 때에 두 분의 만남을 '덕담 나누는 자리' 정도로 평가하는 것에 대해서도 쉽게 동의하기 어렵다"고 지적했습니다.
voice2.mp3


('3번째 뉴스입니다.김은혜 당선인 대변인 [사진 제공: 연합뉴스]윤석열 당선인 측이 문재인 대통령을 겨냥해 "윤석열 당선인의 판단에 마치 문제가 있고 참모들이 당선인의 판단을 흐리는 것처럼 언급하신 것은 대단히 유감스럽다"고 밝혔습니다.\n김은혜 당선인 대변인은 서면 브리핑을 통해 "박수현 소통수석을 통해 전달된 문재인 대통령의 말씀과 관련해 문의가 많아 말씀드린다"며 이같이 말했습니다.\n김 대변인은 "정부 인수인계가 원활치 않은 상황에서, 코로나19와 경제위기 대응이 긴요한 때에 두 분의 만남을 \'덕담 나누는 자리\' 정도로 평가하는 것에 대해서도 쉽게 동의하기 어렵다"고 지적했습니다.',
 'voice2.mp3')

In [16]:
import os
import sys
import requests
import json
import urllib.request
# import pandas as pd
# news_df = pd.read_excel('./news_df.xlsx')
news_list=[]
ff
def process_b(news_df):
    len(news_df.title)
    client_id = "qj40wl5n0t"
    client_secret = "1xzXEYgwlsbv9RQxd1onUSByiyvtBDPFV9NEY1mv"

    for i in range(len(news_df.title)):
        headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret,
        "Content-Type": "application/json"
    }
        language = "ko" # Language of document (ko, ja )
        model = "news" # Model used for summaries (general, news)
        tone = "2" # Converts the tone of the summarized result. (0, 1, 2, 3)
        summaryCount = "3" # This is the number of sentences for the summarized document.
        url= "https://naveropenapi.apigw.ntruss.com/text-summary/v1/summarize" 
        title= ""
        content = news_df.text[i+1]
        data = {
            "document": {
            "title": title,
            "content" : content
            },
            "option": {
            "language": language,
            "model": model,
            "tone": tone,
            "summaryCount" : summaryCount
            }
        }
        # print(json.dumps(data, indent=4, sort_keys=True))
        response = requests.post(url, data=json.dumps(data), headers=headers)
        rescode = response.status_code
        if(rescode == 200):
            r1=response.text  
            # print(r1)

        else:
            print("Error : " + response.text)
    
        # r1 = json.loads(r1)
        r1d = json.loads(r1)
        # r1 = json.dumps(r1)
        # r1r1= r1['summary']
        r1r1 = r1d['summary']
        #r1r1= r1.split('연합뉴스')[1]
        r1r1 = '{}번째 뉴스입니다.{}'.format(i+1, r1r1)
        news_list.append(r1r1)
        #print(r1r1)
        print(news_list)
    
        encText = urllib.parse.quote(r1r1)
        data = "speaker=njihun&volume=0&speed=0&pitch=0&format=mp3&text=" + encText;
        url = "https://naveropenapi.apigw.ntruss.com/tts-premium/v1/tts"
        request = urllib.request.Request(url)
        request.add_header("X-NCP-APIGW-API-KEY-ID",client_id)
        request.add_header("X-NCP-APIGW-API-KEY",client_secret)
        response = urllib.request.urlopen(request, data=data.encode('utf-8'))
        rescode = response.getcode()
        if(rescode==200):
            response_body = response.read()
            fn = 'voice{}.mp3'.format(i)
            ff.append(fn)
            with open(fn, 'wb') as f:
                f.write(response_body)
                
            print(fn)
        else:
            print("Error Code:" + rescode)
            
    result = (r1r1, fn)
    
    return news_list

NameError: name 'ff' is not defined

In [79]:
ff

[]

In [58]:
news_list = process_b(news_df)
news_list

['1번째 뉴스입니다.북한이 대륙간 탄도미사일, ICBM을 시험 발사한 것과 관련해 문재인 대통령은 "김정은 국무위원장이 국제사회에 약속한 발사 유예를 스스로 파기한 것"이라며 "강력히 규탄한다"고 밝혔습니다.\n문 대통령은 오늘 오후 청와대에서 국가안전보장회의, NSC 회의를 긴급 소집해 "북한의 이번 발사는 한반도와 지역·국제사회에 심각한 위협을 야기하고 유엔안보리 결의를 명백히 위반한 것"이라며 이같이 말했습니다.\n이어 "정부 교체기 안보에 한 치의 빈틈도 없도록 한미동맹을 바탕으로 국제사회와 협력해 모든 대응 조치를 강구하라"고 지시하고, 특히 "대통령 당선인 측과도 긴밀하게 협력하라"고 당부했습니다.']
voice0.mp3
['1번째 뉴스입니다.북한이 대륙간 탄도미사일, ICBM을 시험 발사한 것과 관련해 문재인 대통령은 "김정은 국무위원장이 국제사회에 약속한 발사 유예를 스스로 파기한 것"이라며 "강력히 규탄한다"고 밝혔습니다.\n문 대통령은 오늘 오후 청와대에서 국가안전보장회의, NSC 회의를 긴급 소집해 "북한의 이번 발사는 한반도와 지역·국제사회에 심각한 위협을 야기하고 유엔안보리 결의를 명백히 위반한 것"이라며 이같이 말했습니다.\n이어 "정부 교체기 안보에 한 치의 빈틈도 없도록 한미동맹을 바탕으로 국제사회와 협력해 모든 대응 조치를 강구하라"고 지시하고, 특히 "대통령 당선인 측과도 긴밀하게 협력하라"고 당부했습니다.', '2번째 뉴스입니다.사진 제공: 연합뉴스]문재인 대통령이 박근혜 전 대통령에게 \'늘 건강하십시오\'라는 문구가 적힌 퇴원 축하 난을 보냈다고, 청와대가 밝혔습니다.\n또 축하난을 받은 박 전 대통령은 문 대통령에게 "마무리 잘 하시고, 건강을 잘 챙기시길 바란다"는 말을 전해 왔다고 청와대는 설명했습니다.\n청와대는 "문 대통령의 축하난은 지난 22일 오후, 김한규 청와대 정무비서관이 유영하 변호사를 통해 박 전 대통령에게 전달했다"고 덧붙였습니다.']
voice1.mp3
['1번째 뉴스입니다.북한이 대륙간 탄도미사일

['1번째 뉴스입니다.북한이 대륙간 탄도미사일, ICBM을 시험 발사한 것과 관련해 문재인 대통령은 "김정은 국무위원장이 국제사회에 약속한 발사 유예를 스스로 파기한 것"이라며 "강력히 규탄한다"고 밝혔습니다.\n문 대통령은 오늘 오후 청와대에서 국가안전보장회의, NSC 회의를 긴급 소집해 "북한의 이번 발사는 한반도와 지역·국제사회에 심각한 위협을 야기하고 유엔안보리 결의를 명백히 위반한 것"이라며 이같이 말했습니다.\n이어 "정부 교체기 안보에 한 치의 빈틈도 없도록 한미동맹을 바탕으로 국제사회와 협력해 모든 대응 조치를 강구하라"고 지시하고, 특히 "대통령 당선인 측과도 긴밀하게 협력하라"고 당부했습니다.',
 '2번째 뉴스입니다.사진 제공: 연합뉴스]문재인 대통령이 박근혜 전 대통령에게 \'늘 건강하십시오\'라는 문구가 적힌 퇴원 축하 난을 보냈다고, 청와대가 밝혔습니다.\n또 축하난을 받은 박 전 대통령은 문 대통령에게 "마무리 잘 하시고, 건강을 잘 챙기시길 바란다"는 말을 전해 왔다고 청와대는 설명했습니다.\n청와대는 "문 대통령의 축하난은 지난 22일 오후, 김한규 청와대 정무비서관이 유영하 변호사를 통해 박 전 대통령에게 전달했다"고 덧붙였습니다.',
 '3번째 뉴스입니다.김은혜 당선인 대변인 [사진 제공: 연합뉴스]윤석열 당선인 측이 문재인 대통령을 겨냥해 "윤석열 당선인의 판단에 마치 문제가 있고 참모들이 당선인의 판단을 흐리는 것처럼 언급하신 것은 대단히 유감스럽다"고 밝혔습니다.\n김은혜 당선인 대변인은 서면 브리핑을 통해 "박수현 소통수석을 통해 전달된 문재인 대통령의 말씀과 관련해 문의가 많아 말씀드린다"며 이같이 말했습니다.\n김 대변인은 "정부 인수인계가 원활치 않은 상황에서, 코로나19와 경제위기 대응이 긴요한 때에 두 분의 만남을 \'덕담 나누는 자리\' 정도로 평가하는 것에 대해서도 쉽게 동의하기 어렵다"고 지적했습니다.']

In [65]:
news_list[0].replace('\n','')

'1번째 뉴스입니다.북한이 대륙간 탄도미사일, ICBM을 시험 발사한 것과 관련해 문재인 대통령은 "김정은 국무위원장이 국제사회에 약속한 발사 유예를 스스로 파기한 것"이라며 "강력히 규탄한다"고 밝혔습니다.문 대통령은 오늘 오후 청와대에서 국가안전보장회의, NSC 회의를 긴급 소집해 "북한의 이번 발사는 한반도와 지역·국제사회에 심각한 위협을 야기하고 유엔안보리 결의를 명백히 위반한 것"이라며 이같이 말했습니다.이어 "정부 교체기 안보에 한 치의 빈틈도 없도록 한미동맹을 바탕으로 국제사회와 협력해 모든 대응 조치를 강구하라"고 지시하고, 특히 "대통령 당선인 측과도 긴밀하게 협력하라"고 당부했습니다.'

In [7]:
#!/usr/bin/env python3
#-*- codig: utf-8 -*-
import sys
import requests
import json

client_id = "qj40wl5n0t"
client_secret = "1xzXEYgwlsbv9RQxd1onUSByiyvtBDPFV9NEY1mv"

headers = {
    "X-NCP-APIGW-API-KEY-ID": client_id,
    "X-NCP-APIGW-API-KEY": client_secret,
    "Content-Type": "application/json"
}

language = "ko" # Language of document (ko, ja )
model = "news" # Model used for summaries (general, news)
tone = "2" # Converts the tone of the summarized result. (0, 1, 2, 3)
summaryCount = "3" # This is the number of sentences for the summarized document.
url= "https://naveropenapi.apigw.ntruss.com/text-summary/v1/summarize" 
title= "'하루 2000억' 판 커지는 간편송금 시장"
content = "간편송금 이용금액이 하루 평균 2000억원을 넘어섰다. 한국은행이 17일 발표한 '2019년 상반기중 전자지급서비스 이용 현황'에 따르면 올해 상반기 간편송금서비스 이용금액(일평균)은 지난해 하반기 대비 60.7% 증가한 2005억원으로 집계됐다. 같은 기간 이용건수(일평균)는 34.8% 늘어난 218만건이었다. 간편 송금 시장에는 선불전자지급서비스를 제공하는 전자금융업자와 금융기관 등이 참여하고 있다. 이용금액은 전자금융업자가 하루평균 1879억원, 금융기관이 126억원이었다. 한은은 카카오페이, 토스 등 간편송금 서비스를 제공하는 업체 간 경쟁이 심화되면서 이용규모가 크게 확대됐다고 분석했다. 국회 정무위원회 소속 바른미래당 유의동 의원에 따르면 카카오페이, 토스 등 선불전자지급서비스 제공업체는 지난해 마케팅 비용으로 1000억원 이상을 지출했다. 마케팅 비용 지출규모는 카카오페이가 491억원, 비바리퍼블리카(토스)가 134억원 등 순으로 많았다."


data = {
    "document": {
    "title": title,
    "content" : content
    },
    "option": {
    "language": language,
    "model": model,
    "tone": tone,
    "summaryCount" : summaryCount
    }
}

print(json.dumps(data, indent=4, sort_keys=True))
response = requests.post(url, data=json.dumps(data), headers=headers)
rescode = response.status_code
if(rescode == 200):
    print (response.text)
else:
    print("Error : " + response.text)


{
    "document": {
        "content": "\uac04\ud3b8\uc1a1\uae08 \uc774\uc6a9\uae08\uc561\uc774 \ud558\ub8e8 \ud3c9\uade0 2000\uc5b5\uc6d0\uc744 \ub118\uc5b4\uc130\ub2e4. \ud55c\uad6d\uc740\ud589\uc774 17\uc77c \ubc1c\ud45c\ud55c '2019\ub144 \uc0c1\ubc18\uae30\uc911 \uc804\uc790\uc9c0\uae09\uc11c\ube44\uc2a4 \uc774\uc6a9 \ud604\ud669'\uc5d0 \ub530\ub974\uba74 \uc62c\ud574 \uc0c1\ubc18\uae30 \uac04\ud3b8\uc1a1\uae08\uc11c\ube44\uc2a4 \uc774\uc6a9\uae08\uc561(\uc77c\ud3c9\uade0)\uc740 \uc9c0\ub09c\ud574 \ud558\ubc18\uae30 \ub300\ube44 60.7% \uc99d\uac00\ud55c 2005\uc5b5\uc6d0\uc73c\ub85c \uc9d1\uacc4\ub410\ub2e4. \uac19\uc740 \uae30\uac04 \uc774\uc6a9\uac74\uc218(\uc77c\ud3c9\uade0)\ub294 34.8% \ub298\uc5b4\ub09c 218\ub9cc\uac74\uc774\uc5c8\ub2e4. \uac04\ud3b8 \uc1a1\uae08 \uc2dc\uc7a5\uc5d0\ub294 \uc120\ubd88\uc804\uc790\uc9c0\uae09\uc11c\ube44\uc2a4\ub97c \uc81c\uacf5\ud558\ub294 \uc804\uc790\uae08\uc735\uc5c5\uc790\uc640 \uae08\uc735\uae30\uad00 \ub4f1\uc774 \ucc38\uc5ec\ud558\uace0 \

In [122]:
import os
import sys
import requests
import json
import urllib.request
# import pandas as pd
# news_df = pd.read_excel('./news_df.xlsx')
news_list=[]
ff=[]

def process_b(news_df):
    len(news_df.title)
    client_id = "qj40wl5n0t"
    client_secret = "1xzXEYgwlsbv9RQxd1onUSByiyvtBDPFV9NEY1mv"

    for i in range(len(news_df.title)):
        headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret,
        "Content-Type": "application/json"
    }
        language = "ko" # Language of document (ko, ja )
        model = "news" # Model used for summaries (general, news)
        tone = "2" # Converts the tone of the summarized result. (0, 1, 2, 3)
        summaryCount = "3" # This is the number of sentences for the summarized document.
        url= "https://naveropenapi.apigw.ntruss.com/text-summary/v1/summarize" 
        title= ""
        content = news_df.text[i+1]
        data = {
            "document": {
            "title": title,
            "content" : content
            },
            "option": {
            "language": language,
            "model": model,
            "tone": tone,
            "summaryCount" : summaryCount
            }
        }
        # print(json.dumps(data, indent=4, sort_keys=True))
        response = requests.post(url, data=json.dumps(data), headers=headers)
        rescode = response.status_code
        if(rescode == 200):
            r1=response.text  
            print(r1)

        else:
            print("Error : " + response.text)

        r1

        r1d = json.loads(r1)
        r1r1 = r1d['summary']
    #    r1r1= r1.split('연합뉴스')[1]
        r1r1 = '{}번째 뉴스입니다. {} '.format(i+1, r1r1)
        news_list.append(r1r1)
        print(r1r1)
        print(news_list)            
    
        encText = urllib.parse.quote(r1r1)
        data = "speaker=njihun&volume=0&speed=0&pitch=0&format=mp3&text=" + encText;
        url = "https://naveropenapi.apigw.ntruss.com/tts-premium/v1/tts"
        request = urllib.request.Request(url)
        request.add_header("X-NCP-APIGW-API-KEY-ID",client_id)
        request.add_header("X-NCP-APIGW-API-KEY",client_secret)
        response = urllib.request.urlopen(request, data=data.encode('utf-8'))
        rescode = response.getcode()
        if(rescode==200):
            response_body = response.read()
            fn = 'voice{}.mp3'.format(i)
            ff
            with open(fn, 'wb') as f:
                f.write(response_body)
            print(fn)
        else:
            print("Error Code:" + rescode)
            
    result = (r1r1, fn)
    
    return news_list

In [118]:
r1r1 = process_b(news_df)
r1r1

{"summary":"지난 27일 오전 서울역 광장에 설치된 코로나19 임시선별 검사소를 찾은 시민들이 검사를 기다리고 있습니다.\n다음달 3일 현행 사회적 거리두기가 끝나는 가운데 거리두기를 어떻게 조정할지는 다음달 1일 열리는 중대본 회의에서 결정될 것으로 보입니다.\n정부가 방역과 의료체계 등 상황에 따라 거리두기 완화 조치를 본격 검토하겠다고 밝힌 만큼 이번주 확진자수 추이에 따라 각종 거리두기 지침의 대폭 완화 가능성이 있습니다."}
1번째 뉴스입니다. 지난 27일 오전 서울역 광장에 설치된 코로나19 임시선별 검사소를 찾은 시민들이 검사를 기다리고 있습니다.
다음달 3일 현행 사회적 거리두기가 끝나는 가운데 거리두기를 어떻게 조정할지는 다음달 1일 열리는 중대본 회의에서 결정될 것으로 보입니다.
정부가 방역과 의료체계 등 상황에 따라 거리두기 완화 조치를 본격 검토하겠다고 밝힌 만큼 이번주 확진자수 추이에 따라 각종 거리두기 지침의 대폭 완화 가능성이 있습니다. 
['1번째 뉴스입니다. 지난 27일 오전 서울역 광장에 설치된 코로나19 임시선별 검사소를 찾은 시민들이 검사를 기다리고 있습니다.\n다음달 3일 현행 사회적 거리두기가 끝나는 가운데 거리두기를 어떻게 조정할지는 다음달 1일 열리는 중대본 회의에서 결정될 것으로 보입니다.\n정부가 방역과 의료체계 등 상황에 따라 거리두기 완화 조치를 본격 검토하겠다고 밝힌 만큼 이번주 확진자수 추이에 따라 각종 거리두기 지침의 대폭 완화 가능성이 있습니다. ']
voice0.mp3
{"summary":"지난 26일 오전 중구 서울역 광장 코로나19 임시 선별검사소를 찾은 시민들이 검사를 기다리고 있습니다.\n오늘(27일) 질병관리청 중앙방역대책본부는 이날 0시 기준 코로나19 신규 확진자가 31만8130명 발생했다고 밝혔습니다.\n신규 확진자 가운데 국내 발생 환자는 31만8087명입니다."}
2번째 뉴스입니다. 지난 26일 오전 중구 서울역 광장 코로나19 임시 선별검사소를 찾은 시민들이 검사를 기다리고

['1번째 뉴스입니다. 지난 27일 오전 서울역 광장에 설치된 코로나19 임시선별 검사소를 찾은 시민들이 검사를 기다리고 있습니다.\n다음달 3일 현행 사회적 거리두기가 끝나는 가운데 거리두기를 어떻게 조정할지는 다음달 1일 열리는 중대본 회의에서 결정될 것으로 보입니다.\n정부가 방역과 의료체계 등 상황에 따라 거리두기 완화 조치를 본격 검토하겠다고 밝힌 만큼 이번주 확진자수 추이에 따라 각종 거리두기 지침의 대폭 완화 가능성이 있습니다. ',
 '2번째 뉴스입니다. 지난 26일 오전 중구 서울역 광장 코로나19 임시 선별검사소를 찾은 시민들이 검사를 기다리고 있습니다.\n오늘(27일) 질병관리청 중앙방역대책본부는 이날 0시 기준 코로나19 신규 확진자가 31만8130명 발생했다고 밝혔습니다.\n신규 확진자 가운데 국내 발생 환자는 31만8087명입니다. ',
 '3번째 뉴스입니다. 자료사진=JTBC 방송화면 캡처〉 인천에서 신종 코로나바이러스 감염증(코로나19)에 확진된 산모가 수용 가능 병원을 찾지 못해 구급차에서 출산하는 일이 발생했습니다.\n오늘(25일) 인천소방본부는 인천 서구 청라동에 거주하는 산모 A씨가 어제(24일) 구급차에서 남자아이를 출산했다고 JTBC에 밝혔습니다.\nA씨는 어제 오후 6시 20분쯤 자택에서 진통이 온다며 119구급대에 신고를 했습니다. ']

In [78]:
news_list = process_b(news_df)
news_list

{"summary":"지난 27일 오전 서울역 광장에 설치된 코로나19 임시선별 검사소를 찾은 시민들이 검사를 기다리고 있습니다.\n다음달 3일 현행 사회적 거리두기가 끝나는 가운데 거리두기를 어떻게 조정할지는 다음달 1일 열리는 중대본 회의에서 결정될 것으로 보입니다.\n정부가 방역과 의료체계 등 상황에 따라 거리두기 완화 조치를 본격 검토하겠다고 밝힌 만큼 이번주 확진자수 추이에 따라 각종 거리두기 지침의 대폭 완화 가능성이 있습니다."}
1번째 뉴스입니다. 지난 27일 오전 서울역 광장에 설치된 코로나19 임시선별 검사소를 찾은 시민들이 검사를 기다리고 있습니다.
다음달 3일 현행 사회적 거리두기가 끝나는 가운데 거리두기를 어떻게 조정할지는 다음달 1일 열리는 중대본 회의에서 결정될 것으로 보입니다.
정부가 방역과 의료체계 등 상황에 따라 거리두기 완화 조치를 본격 검토하겠다고 밝힌 만큼 이번주 확진자수 추이에 따라 각종 거리두기 지침의 대폭 완화 가능성이 있습니다. 
['1번째 뉴스입니다. 지난 27일 오전 서울역 광장에 설치된 코로나19 임시선별 검사소를 찾은 시민들이 검사를 기다리고 있습니다.\n다음달 3일 현행 사회적 거리두기가 끝나는 가운데 거리두기를 어떻게 조정할지는 다음달 1일 열리는 중대본 회의에서 결정될 것으로 보입니다.\n정부가 방역과 의료체계 등 상황에 따라 거리두기 완화 조치를 본격 검토하겠다고 밝힌 만큼 이번주 확진자수 추이에 따라 각종 거리두기 지침의 대폭 완화 가능성이 있습니다. ']
voice0.mp3
{"summary":"지난 26일 오전 중구 서울역 광장 코로나19 임시 선별검사소를 찾은 시민들이 검사를 기다리고 있습니다.\n오늘(27일) 질병관리청 중앙방역대책본부는 이날 0시 기준 코로나19 신규 확진자가 31만8130명 발생했다고 밝혔습니다.\n신규 확진자 가운데 국내 발생 환자는 31만8087명입니다."}
2번째 뉴스입니다. 지난 26일 오전 중구 서울역 광장 코로나19 임시 선별검사소를 찾은 시민들이 검사를 기다리고

['1번째 뉴스입니다. 지난 27일 오전 서울역 광장에 설치된 코로나19 임시선별 검사소를 찾은 시민들이 검사를 기다리고 있습니다.\n다음달 3일 현행 사회적 거리두기가 끝나는 가운데 거리두기를 어떻게 조정할지는 다음달 1일 열리는 중대본 회의에서 결정될 것으로 보입니다.\n정부가 방역과 의료체계 등 상황에 따라 거리두기 완화 조치를 본격 검토하겠다고 밝힌 만큼 이번주 확진자수 추이에 따라 각종 거리두기 지침의 대폭 완화 가능성이 있습니다. ',
 '2번째 뉴스입니다. 지난 26일 오전 중구 서울역 광장 코로나19 임시 선별검사소를 찾은 시민들이 검사를 기다리고 있습니다.\n오늘(27일) 질병관리청 중앙방역대책본부는 이날 0시 기준 코로나19 신규 확진자가 31만8130명 발생했다고 밝혔습니다.\n신규 확진자 가운데 국내 발생 환자는 31만8087명입니다. ',
 '3번째 뉴스입니다. 자료사진=JTBC 방송화면 캡처〉 인천에서 신종 코로나바이러스 감염증(코로나19)에 확진된 산모가 수용 가능 병원을 찾지 못해 구급차에서 출산하는 일이 발생했습니다.\n오늘(25일) 인천소방본부는 인천 서구 청라동에 거주하는 산모 A씨가 어제(24일) 구급차에서 남자아이를 출산했다고 JTBC에 밝혔습니다.\nA씨는 어제 오후 6시 20분쯤 자택에서 진통이 온다며 119구급대에 신고를 했습니다. ']

In [81]:
### 요약부분 수정하는 함수 
def clean_news(news_list):
    remove = ['\n', '\\', '[사진 제공: 연합뉴스]','자료사진=', '캡처','=']
    clean_list = []
    for t in remove:
        for i, o in enumerate(news_list):
            news_list[i] = o.replace(t, "")

    return news_list

In [82]:
news_list = clean_news(news_list) 
news_list

['1번째 뉴스입니다. 지난 27일 오전 서울역 광장에 설치된 코로나19 임시선별 검사소를 찾은 시민들이 검사를 기다리고 있습니다.다음달 3일 현행 사회적 거리두기가 끝나는 가운데 거리두기를 어떻게 조정할지는 다음달 1일 열리는 중대본 회의에서 결정될 것으로 보입니다.정부가 방역과 의료체계 등 상황에 따라 거리두기 완화 조치를 본격 검토하겠다고 밝힌 만큼 이번주 확진자수 추이에 따라 각종 거리두기 지침의 대폭 완화 가능성이 있습니다. ',
 '2번째 뉴스입니다. 지난 26일 오전 중구 서울역 광장 코로나19 임시 선별검사소를 찾은 시민들이 검사를 기다리고 있습니다.오늘(27일) 질병관리청 중앙방역대책본부는 이날 0시 기준 코로나19 신규 확진자가 31만8130명 발생했다고 밝혔습니다.신규 확진자 가운데 국내 발생 환자는 31만8087명입니다. ',
 '3번째 뉴스입니다. JTBC 방송화면 〉 인천에서 신종 코로나바이러스 감염증(코로나19)에 확진된 산모가 수용 가능 병원을 찾지 못해 구급차에서 출산하는 일이 발생했습니다.오늘(25일) 인천소방본부는 인천 서구 청라동에 거주하는 산모 A씨가 어제(24일) 구급차에서 남자아이를 출산했다고 JTBC에 밝혔습니다.A씨는 어제 오후 6시 20분쯤 자택에서 진통이 온다며 119구급대에 신고를 했습니다. ']

In [69]:
clean_list = clean_news(news_list)
clean_list

IndexError: list assignment index out of range

In [19]:
news_list[0].replace('\n','')

'1번째 뉴스입니다. 지난 27일 오전 서울역 광장에 설치된 코로나19 임시선별 검사소를 찾은 시민들이 검사를 기다리고 있습니다.다음달 3일 현행 사회적 거리두기가 끝나는 가운데 거리두기를 어떻게 조정할지는 다음달 1일 열리는 중대본 회의에서 결정될 것으로 보입니다.정부가 방역과 의료체계 등 상황에 따라 거리두기 완화 조치를 본격 검토하겠다고 밝힌 만큼 이번주 확진자수 추이에 따라 각종 거리두기 지침의 대폭 완화 가능성이 있습니다. '

In [29]:
type(fn)

str

In [119]:
### 요약부분 수정하는 함수 -> 문자열로 합치기
def clean_news(news_list):
    remove = ['\n', '\\', '[사진 제공: 연합뉴스]' ]
    
    for t in remove:
        for i, o in enumerate(news_list):
            news_list[i] = o.replace(t, "")
            
    summary = news_list.append('/n')
    return summary


In [120]:
summary = clean_news(news_list)
summary

In [109]:
### 요약부분 수정하는 함수 -> 문자열로 합치기
def clean_news(news_list):
    remove = ['\n', '\\', '[사진 제공: 연합뉴스]' ]
    
    for t in remove:
        for i, o in enumerate(news_list):
            news_list[i] = o.replace(t, "")
            
    summary=join('\n'.news_list)
    return summary


In [121]:
summary

In [ ]:
from fastapi.responses import PlainTextResponse
@app_fastapi.get("/get_log", response_class=PlainTextResponse)
async def get_log():
    return clean_news(news_list)


In [128]:
import os
import sys
import requests
import json
import urllib.request
# import pandas as pd
# news_df = pd.read_excel('./news_df.xlsx')
news_list=[]
site=[]
def process_b(news_df):
    len(news_df.title)
    client_id = "qj40wl5n0t"
    client_secret = "1xzXEYgwlsbv9RQxd1onUSByiyvtBDPFV9NEY1mv"

    for i in range(len(news_df.title)):
        headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret,
        "Content-Type": "application/json"
    }
        language = "ko" # Language of document (ko, ja )
        model = "news" # Model used for summaries (general, news)
        tone = "2" # Converts the tone of the summarized result. (0, 1, 2, 3)
        summaryCount = "3" # This is the number of sentences for the summarized document.
        url= "https://naveropenapi.apigw.ntruss.com/text-summary/v1/summarize" 
        title = ""
        content = news_df.text[i+1]
        data = {
            "document": {
            "title": title,
            "content" : content},
            "option": {
            "language": language,
            "model": model,
            "tone": tone,
            "summaryCount" : summaryCount
            }
        }
        # print(json.dumps(data, indent=4, sort_keys=True))
        response = requests.post(url, data=json.dumps(data), headers=headers)
        rescode = response.status_code
        if(rescode == 200):
            r1=response.text  
            print(r1)

        else:
            print("Error : " + response.text)

        r1
        site = news_df.url[i+1]

        r1d = json.loads(r1)
        r1r1 = r1d['summary']
    #    r1r1= r1.split('연합뉴스')[1]
        r1r1 = '{}번째 뉴스입니다. {} '.format(i+1, r1r1)
        news_list.append(r1r1)
        news_list.append(site)
        print(r1r1)
        print(news_list)            
    
        encText = urllib.parse.quote(r1r1)
        data = "speaker=njihun&volume=0&speed=0&pitch=0&format=mp3&text=" + encText;
        url = "https://naveropenapi.apigw.ntruss.com/tts-premium/v1/tts"
        request = urllib.request.Request(url)
        request.add_header("X-NCP-APIGW-API-KEY-ID",client_id)
        request.add_header("X-NCP-APIGW-API-KEY",client_secret)
        response = urllib.request.urlopen(request, data=data.encode('utf-8'))
        rescode = response.getcode()
        if(rescode==200):
            response_body = response.read()
            fn = 'voice{}.mp3'.format(i)
            with open(fn, 'wb') as f:
                f.write(response_body)
            print(fn)
        else:
            print("Error Code:" + rescode)
            
    result = (r1r1, fn)
    
    return news_list

In [129]:
site = news_df.url[i+1]
site

'https://news.jtbc.joins.com/article/article.aspx?news_id=NB12052508'

In [130]:
news_list = process_b(news_df)
news_list

{"summary":"지난 27일 오전 서울역 광장에 설치된 코로나19 임시선별 검사소를 찾은 시민들이 검사를 기다리고 있습니다.\n다음달 3일 현행 사회적 거리두기가 끝나는 가운데 거리두기를 어떻게 조정할지는 다음달 1일 열리는 중대본 회의에서 결정될 것으로 보입니다.\n정부가 방역과 의료체계 등 상황에 따라 거리두기 완화 조치를 본격 검토하겠다고 밝힌 만큼 이번주 확진자수 추이에 따라 각종 거리두기 지침의 대폭 완화 가능성이 있습니다."}
1번째 뉴스입니다. 지난 27일 오전 서울역 광장에 설치된 코로나19 임시선별 검사소를 찾은 시민들이 검사를 기다리고 있습니다.
다음달 3일 현행 사회적 거리두기가 끝나는 가운데 거리두기를 어떻게 조정할지는 다음달 1일 열리는 중대본 회의에서 결정될 것으로 보입니다.
정부가 방역과 의료체계 등 상황에 따라 거리두기 완화 조치를 본격 검토하겠다고 밝힌 만큼 이번주 확진자수 추이에 따라 각종 거리두기 지침의 대폭 완화 가능성이 있습니다. 
['1번째 뉴스입니다. 지난 27일 오전 서울역 광장에 설치된 코로나19 임시선별 검사소를 찾은 시민들이 검사를 기다리고 있습니다.\n다음달 3일 현행 사회적 거리두기가 끝나는 가운데 거리두기를 어떻게 조정할지는 다음달 1일 열리는 중대본 회의에서 결정될 것으로 보입니다.\n정부가 방역과 의료체계 등 상황에 따라 거리두기 완화 조치를 본격 검토하겠다고 밝힌 만큼 이번주 확진자수 추이에 따라 각종 거리두기 지침의 대폭 완화 가능성이 있습니다. ', 'https://news.jtbc.joins.com/article/article.aspx?news_id=NB12052672']
voice0.mp3
{"summary":"지난 26일 오전 중구 서울역 광장 코로나19 임시 선별검사소를 찾은 시민들이 검사를 기다리고 있습니다.\n오늘(27일) 질병관리청 중앙방역대책본부는 이날 0시 기준 코로나19 신규 확진자가 31만8130명 발생했다고 밝혔습니다.\n신규 확진자 가운데 국내 발생 환자는 31만8087

['1번째 뉴스입니다. 지난 27일 오전 서울역 광장에 설치된 코로나19 임시선별 검사소를 찾은 시민들이 검사를 기다리고 있습니다.\n다음달 3일 현행 사회적 거리두기가 끝나는 가운데 거리두기를 어떻게 조정할지는 다음달 1일 열리는 중대본 회의에서 결정될 것으로 보입니다.\n정부가 방역과 의료체계 등 상황에 따라 거리두기 완화 조치를 본격 검토하겠다고 밝힌 만큼 이번주 확진자수 추이에 따라 각종 거리두기 지침의 대폭 완화 가능성이 있습니다. ',
 'https://news.jtbc.joins.com/article/article.aspx?news_id=NB12052672',
 '2번째 뉴스입니다. 지난 26일 오전 중구 서울역 광장 코로나19 임시 선별검사소를 찾은 시민들이 검사를 기다리고 있습니다.\n오늘(27일) 질병관리청 중앙방역대책본부는 이날 0시 기준 코로나19 신규 확진자가 31만8130명 발생했다고 밝혔습니다.\n신규 확진자 가운데 국내 발생 환자는 31만8087명입니다. ',
 'https://news.jtbc.joins.com/article/article.aspx?news_id=NB12052598',
 '3번째 뉴스입니다. 자료사진=JTBC 방송화면 캡처〉 인천에서 신종 코로나바이러스 감염증(코로나19)에 확진된 산모가 수용 가능 병원을 찾지 못해 구급차에서 출산하는 일이 발생했습니다.\n오늘(25일) 인천소방본부는 인천 서구 청라동에 거주하는 산모 A씨가 어제(24일) 구급차에서 남자아이를 출산했다고 JTBC에 밝혔습니다.\nA씨는 어제 오후 6시 20분쯤 자택에서 진통이 온다며 119구급대에 신고를 했습니다. ',
 'https://news.jtbc.joins.com/article/article.aspx?news_id=NB12052508']

In [139]:
import os
import sys
import requests
import json
import urllib.request
# import pandas as pd
# news_df = pd.read_excel('./news_df.xlsx')
news_list=[]
site=[]
def process_b(news_df):
    len(news_df.title)
    client_id = "qj40wl5n0t"
    client_secret = "1xzXEYgwlsbv9RQxd1onUSByiyvtBDPFV9NEY1mv"

    for i in range(len(news_df.title)):
        headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret,
        "Content-Type": "application/json"
    }
        language = "ko" # Language of document (ko, ja )
        model = "news" # Model used for summaries (general, news)
        tone = "2" # Converts the tone of the summarized result. (0, 1, 2, 3)
        summaryCount = "3" # This is the number of sentences for the summarized document.
        url= "https://naveropenapi.apigw.ntruss.com/text-summary/v1/summarize" 
        title = ""
        content = news_df.text[i+1]
        data = {
            "document": {
            "title": title,
            "content" : content},
            "option": {
            "language": language,
            "model": model,
            "tone": tone,
            "summaryCount" : summaryCount
            }
        }
        # print(json.dumps(data, indent=4, sort_keys=True))
        response = requests.post(url, data=json.dumps(data), headers=headers)
        rescode = response.status_code
        if(rescode == 200):
            r1=response.text  
            print(r1)

        else:
            print("Error : " + response.text)

        r1
        site = news_df.url[i+1]

        r1d = json.loads(r1)
        r1r1 = r1d['summary']
    #    r1r1= r1.split('연합뉴스')[1]
        r1r1 = '{}번째 뉴스입니다. {} '.format(i+1, r1r1)
        news_list.append(r1r1)
        news_list.append(site)
        print(r1r1)
        print(news_list)            
    
        encText = urllib.parse.quote(r1r1)
        data = "speaker=njihun&volume=0&speed=0&pitch=0&format=mp3&text=" + encText;
        url = "https://naveropenapi.apigw.ntruss.com/tts-premium/v1/tts"
        request = urllib.request.Request(url)
        request.add_header("X-NCP-APIGW-API-KEY-ID",client_id)
        request.add_header("X-NCP-APIGW-API-KEY",client_secret)
        response = urllib.request.urlopen(request, data=data.encode('utf-8'))
        rescode = response.getcode()
        if(rescode==200):
            response_body = response.read()
            fn = 'voice{}.mp3'.format(i)
            with open(fn, 'wb') as f:
                f.write(response_body)
            print(fn)
        else:
            print("Error Code:" + rescode)
            
    result = (r1r1, fn)
    
    return news_list

In [140]:
news_list = process_b(news_df)
news_list

{"summary":"지난 27일 오전 서울역 광장에 설치된 코로나19 임시선별 검사소를 찾은 시민들이 검사를 기다리고 있습니다.\n다음달 3일 현행 사회적 거리두기가 끝나는 가운데 거리두기를 어떻게 조정할지는 다음달 1일 열리는 중대본 회의에서 결정될 것으로 보입니다.\n정부가 방역과 의료체계 등 상황에 따라 거리두기 완화 조치를 본격 검토하겠다고 밝힌 만큼 이번주 확진자수 추이에 따라 각종 거리두기 지침의 대폭 완화 가능성이 있습니다."}
1번째 뉴스입니다. 지난 27일 오전 서울역 광장에 설치된 코로나19 임시선별 검사소를 찾은 시민들이 검사를 기다리고 있습니다.
다음달 3일 현행 사회적 거리두기가 끝나는 가운데 거리두기를 어떻게 조정할지는 다음달 1일 열리는 중대본 회의에서 결정될 것으로 보입니다.
정부가 방역과 의료체계 등 상황에 따라 거리두기 완화 조치를 본격 검토하겠다고 밝힌 만큼 이번주 확진자수 추이에 따라 각종 거리두기 지침의 대폭 완화 가능성이 있습니다. 
['1번째 뉴스입니다. 지난 27일 오전 서울역 광장에 설치된 코로나19 임시선별 검사소를 찾은 시민들이 검사를 기다리고 있습니다.\n다음달 3일 현행 사회적 거리두기가 끝나는 가운데 거리두기를 어떻게 조정할지는 다음달 1일 열리는 중대본 회의에서 결정될 것으로 보입니다.\n정부가 방역과 의료체계 등 상황에 따라 거리두기 완화 조치를 본격 검토하겠다고 밝힌 만큼 이번주 확진자수 추이에 따라 각종 거리두기 지침의 대폭 완화 가능성이 있습니다. ', 'https://news.jtbc.joins.com/article/article.aspx?news_id=NB12052672']
voice0.mp3
{"summary":"지난 26일 오전 중구 서울역 광장 코로나19 임시 선별검사소를 찾은 시민들이 검사를 기다리고 있습니다.\n오늘(27일) 질병관리청 중앙방역대책본부는 이날 0시 기준 코로나19 신규 확진자가 31만8130명 발생했다고 밝혔습니다.\n신규 확진자 가운데 국내 발생 환자는 31만8087

['1번째 뉴스입니다. 지난 27일 오전 서울역 광장에 설치된 코로나19 임시선별 검사소를 찾은 시민들이 검사를 기다리고 있습니다.\n다음달 3일 현행 사회적 거리두기가 끝나는 가운데 거리두기를 어떻게 조정할지는 다음달 1일 열리는 중대본 회의에서 결정될 것으로 보입니다.\n정부가 방역과 의료체계 등 상황에 따라 거리두기 완화 조치를 본격 검토하겠다고 밝힌 만큼 이번주 확진자수 추이에 따라 각종 거리두기 지침의 대폭 완화 가능성이 있습니다. ',
 'https://news.jtbc.joins.com/article/article.aspx?news_id=NB12052672',
 '2번째 뉴스입니다. 지난 26일 오전 중구 서울역 광장 코로나19 임시 선별검사소를 찾은 시민들이 검사를 기다리고 있습니다.\n오늘(27일) 질병관리청 중앙방역대책본부는 이날 0시 기준 코로나19 신규 확진자가 31만8130명 발생했다고 밝혔습니다.\n신규 확진자 가운데 국내 발생 환자는 31만8087명입니다. ',
 'https://news.jtbc.joins.com/article/article.aspx?news_id=NB12052598',
 '3번째 뉴스입니다. 자료사진=JTBC 방송화면 캡처〉 인천에서 신종 코로나바이러스 감염증(코로나19)에 확진된 산모가 수용 가능 병원을 찾지 못해 구급차에서 출산하는 일이 발생했습니다.\n오늘(25일) 인천소방본부는 인천 서구 청라동에 거주하는 산모 A씨가 어제(24일) 구급차에서 남자아이를 출산했다고 JTBC에 밝혔습니다.\nA씨는 어제 오후 6시 20분쯤 자택에서 진통이 온다며 119구급대에 신고를 했습니다. ',
 'https://news.jtbc.joins.com/article/article.aspx?news_id=NB12052508']